# Chain-of-Thought (CoT) Loan Application Demo

This Jupyter notebook demonstrates **Chain-of-Thought (CoT) Prompting** in the context of a **loan application**. Our AI agent will "think aloud" step-by-step about an applicant’s financial details and then decide whether to approve or deny the loan.

We structure the code into **four sections**:
1. **Loan Criteria**
2. **LLM Adapter**
3. **Chain-of-Thought Agent**
4. **Main Orchestration**

Unlike ReAct, CoT typically performs the entire multi-step reasoning in **one pass**—the agent forms a chain-of-thought and arrives at the conclusion without additional tool calls mid-prompt.


## Section 1: Loan Criteria
In this section, we store the **business logic** for loan approvals. Here, we define minimum credit score, income thresholds, and a simple check function. In reality, this logic could be more detailed or query external databases.


In [ ]:
LOAN_POLICY = {
    "MIN_CREDIT_SCORE": 650,       # Minimum acceptable credit score
    "MIN_ANNUAL_INCOME": 30000,   # Minimum annual income in dollars
    "MAX_DEBT_TO_INCOME": 0.4     # 40% maximum debt-to-income ratio
}

def basic_loan_eligibility_check(credit_score, annual_income, monthly_debt, monthly_income):
    """
    Returns a dict with high-level checks:
    {
      'credit_score_check': bool,
      'income_check': bool,
      'dti_check': bool
    }
    """
    # Avoid division by zero
    if monthly_income > 0:
        dti = monthly_debt / monthly_income
    else:
        dti = 1.0  # Force fail if monthly_income is zero or negative

    results = {
        "credit_score_check": credit_score >= LOAN_POLICY["MIN_CREDIT_SCORE"],
        "income_check": annual_income >= LOAN_POLICY["MIN_ANNUAL_INCOME"],
        "dti_check": dti <= LOAN_POLICY["MAX_DEBT_TO_INCOME"]
    }
    return results

## Section 2: LLM Adapter
Here, we define a **placeholder function** that simulates an LLM call (`call_llm`). In a real-world situation, this would be replaced with calls to an actual LLM provider (OpenAI, Anthropic, etc.). For demonstration, it returns a hardcoded chain-of-thought and decision.


In [ ]:
def call_llm(prompt, temperature=0.2):
    """
    Simulated LLM call for demonstration.
    Returns a chain-of-thought reasoning plus a final conclusion.
    """
    # In an actual application, you'd call your LLM API here.
    # Below, we hardcode a mock response when we detect a certain instruction.

    if "Please reason step-by-step" in prompt:
        response = (
            "Chain of Thought:\n"
            "1. The applicant's credit score meets the minimum requirement.\n"
            "2. The applicant's annual income is slightly above the minimum.\n"
            "3. The debt-to-income ratio is within acceptable range.\n"
            "Therefore, the loan is approved.\n"
            "Final Answer: Approved\n"
        )
        return response
    else:
        return "No chain of thought generated."

## Section 3: Chain-of-Thought Agent
Here, we implement the **Chain-of-Thought** flow. We combine some straightforward domain checks (`basic_loan_eligibility_check`) with an **LLM prompt** that instructs the AI to produce **step-by-step reasoning** and a **final decision**.

**Key point**: We perform the entire chain-of-thought reasoning in **one pass** of the LLM.


In [ ]:
class LoanCoTAgent:
    def __init__(self):
        self.conversation_log = []  # optional logging

    def process_loan_application(self, credit_score, annual_income, monthly_debt, monthly_income):
        """
        1. Run basic checks.
        2. Prompt the LLM for chain-of-thought reasoning.
        3. Parse and return a final decision.
        """
        # Step 1: Basic checks (domain logic)
        eligibility_checks = basic_loan_eligibility_check(
            credit_score=credit_score,
            annual_income=annual_income,
            monthly_debt=monthly_debt,
            monthly_income=monthly_income
        )

        # Build a context string to help the LLM reason about the data
        context_info = (
            f"Applicant's credit score: {credit_score}\n"
            f"Applicant's annual income: {annual_income}\n"
            f"Applicant's monthly debt: {monthly_debt}\n"
            f"Applicant's monthly income: {monthly_income}\n"
            f"Checks:\n"
            f" - credit_score_check: {eligibility_checks['credit_score_check']}\n"
            f" - income_check: {eligibility_checks['income_check']}\n"
            f" - dti_check: {eligibility_checks['dti_check']}\n"
        )

        # Step 2: Prompt the LLM to produce chain-of-thought reasoning
        prompt = (
            "You are a loan approval AI. You have the following data:\n"
            f"{context_info}\n"
            "Please reason step-by-step and produce a final approval decision. "
            "If all checks are true, you can lean towards approval. Otherwise, deny.\n"
            "Your response should include your chain of thought and a final answer in the form: "
            "'Final Answer: Approved' or 'Final Answer: Denied'.\n"
        )

        llm_response = call_llm(prompt)
        self.conversation_log.append(llm_response)

        # Step 3: Extract final decision from the chain-of-thought
        if "Approved" in llm_response:
            decision = "Approved"
        else:
            decision = "Denied"

        return {
            "chain_of_thought": llm_response,
            "decision": decision
        }

## Section 4: Main Orchestration
Finally, we tie everything together. We **simulate** a loan application, provide data to the agent, and then observe the chain-of-thought and decision.

Feel free to modify the applicant data (credit score, income, debts, etc.) to see different results.


In [ ]:
# Example usage
if __name__ == "__main__":
    # Sample applicant data
    applicant_credit_score = 680
    applicant_annual_income = 40000.0
    applicant_monthly_debt = 800.0
    applicant_monthly_income = 3000.0

    # Initialize our Chain-of-Thought agent
    agent = LoanCoTAgent()

    # Process the loan application
    result = agent.process_loan_application(
        credit_score=applicant_credit_score,
        annual_income=applicant_annual_income,
        monthly_debt=applicant_monthly_debt,
        monthly_income=applicant_monthly_income
    )

    # Print both the chain of thought and the final decision
    print("Chain of Thought + Decision:\n")
    print(result["chain_of_thought"], "\n")
    print(f"Decision: {result['decision']}")

## How to Run
1. **Run all cells** in the notebook.
2. The output from the final cell should show you a chain-of-thought explanation and a decision (Approved or Denied).
3. You can adjust any parameters (credit score, income, debt) in the **Main Orchestration** section to explore different outcomes.

## Key Takeaways
1. **Chain-of-Thought Prompting**: We explicitly ask the LLM to "reason step-by-step" and reveal how it arrives at the decision.
2. **Domain Checks + LLM Reasoning**: We combine Python-based guardrails (`basic_loan_eligibility_check`) with an LLM prompt.
3. **Single Pass**: Unlike ReAct, there's no mid-prompt tool usage. The LLM does a single pass of multi-step reasoning.
4. **Confidentiality**: If needed, you can hide or remove the chain-of-thought from the final user-facing message while still capturing it internally.
5. **Extensibility**: You can add more application data (job history, existing loans, etc.) and expand the logic in the same pattern.
